In [93]:
from dotenv import dotenv_values
import requests
import pandas as pd
import ast
from tqdm.notebook import tqdm
from collections import OrderedDict

In [94]:
# OrderedDict containing key-value pairs of secret keys
config = dotenv_values("../.env")
CENSUS_API_KEY = config['CENSUS_API_KEY']

In [95]:
def get_variable_table_df(year):
    variable_table_url = f'https://api.census.gov/data/{year}/acs/acs1/profile/variables.html'
    v_table = pd.read_html(variable_table_url)
    variable_df = pd.DataFrame(v_table[0])
    variable_df['Label'].replace({"!!": " ", ":": ""}, regex=True, inplace=True)
    return variable_df

In [4]:
def get_male_by_age_index(variable_table):
    start_index = variable_table[variable_table['Label'] == 'Estimate Total Male'].index[0]
    end_index = variable_table[variable_table['Label'] == 'Estimate Total Male 85 years and over'].index[0]
    return start_index, end_index + 1

In [5]:
def get_variable_names(variable_table, indeces):
    total_male_by_age_variables = ",".join(variable_table.iloc[indeces[0]: indeces[1]]['Name'].values)
    return total_male_by_age_variables

In [120]:
def get_query_url(year, variables, state_code="*"):
    # API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
    # Data Dictionary: https://api.census.gov/data.html
    host = 'https://api.census.gov/data'
    year = f'/{year}'
    dataset_acronym = '/acs/acs1/profile'
    g = '?get='
    location = f"&for=county:*&in=state:{state_code}"
    usr_key = f"&key={CENSUS_API_KEY}"
    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
    return query_url

In [98]:
def get_query_text(query_url):
    response = requests.get(query_url)
    return response.text

In [99]:
def get_values_from_response(response_text):
    values = [int(i) for i in ast.literal_eval(response_text)[1]]
    return values

In [100]:
def get_labels(variable_df, indeces):
    labels = [i.replace("!!", " ").replace(":", "") for i in variable_df.iloc[indeces[0]:indeces[1]]['Label'].values]
    return labels

In [101]:
def create_year_pop_dataframe(year, labels, values):
    df = pd.DataFrame({year: {labels[i]: values[i] for i in range(len(labels))}}).reindex(labels)
    return df

In [102]:
def create_male_pop_by_age_df(year):
    v_table = get_variable_table_df(year)
    male_by_age_indeces = get_male_by_age_index(v_table)
    variables = get_variable_names(v_table, male_by_age_indeces)
    query_url = get_query_url(year, variables, state_codes)
    response_text = get_query_text(query_url)
    vals = get_values_from_response(response_text)
    labels = get_labels(v_table, male_by_age_indeces)
    df = create_year_pop_dataframe(year, labels, vals)
    return df

In [129]:
variable_dict = OrderedDict()
variable_dict["NAME"] = "County Name"
variable_dict["DP02_0058PE"] = "Percent_EduAttain_25y_over"
variable_dict["DP02_0088PE"] = "Percent_BornInUS"
variable_dict["DP03_0002PE"] = "Percent_InLaborForce"
variable_dict["DP03_0015PE"] = "Percent_AllParentsInLaborForce"

In [121]:
def get_data(year, state_code, variables):
    v_table = get_variable_table_df(year)
    query_url = get_query_url(year, variables=variables, state_code=state_code)
    response_text = get_query_text(query_url).replace("null", "None")
    response_data = eval(response_text)
    df = pd.DataFrame(response_data[1:], columns = response_data[0])
    df = df.astype({col: float for col in df.columns[1:]})
    # vals = get_values_from_response(response_text)
    # labels = get_labels(v_table, male_by_age_indeces)
    # df = create_year_pop_dataframe(year, labels, vals)
    for col in df.columns:
        if col in variable_dict.keys():
            df = df.rename({col:variable_dict[col]}, axis=1)
    return df

In [112]:
list(variable_dict.values())

['County Name',
 'Percent_EduAttain_25y_over',
 'Percent_BornInUS',
 'Avg_HouseholdSize',
 'Percent_InLaborForce',
 'Percent_AllParentsInLaborForce']

In [145]:
# get data for counties across multiple states in question
years = [2014]
# California, Oregon, Arizona, Nevada
# state_codes = ["06", "41", "04", "32"]
state_codes = ["06", "41", "04", "32"]

output = pd.DataFrame(columns=list(variable_dict.values()) + ["state", "county"])
for year in years:
    for state_code in state_codes:
        test = get_data(year, state_code, "NAME,DP02_0058PE,DP02_0088PE,DP03_0002PE,DP03_0015PE")
        output = pd.concat([output, test], axis=0)


In [144]:
output

,County Name,Percent_EduAttain_25y_over,Percent_BornInUS,Percent_InLaborForce,Percent_AllParentsInLaborForce,state,county
0,"Alameda County, California",1039030.0,67.7,65.9,64.5,6.0,1.0
1,"Butte County, California",140947.0,91.3,55.8,59.5,6.0,7.0
2,"Contra Costa County, California",716292.0,75.5,65.4,63.5,6.0,13.0
3,"El Dorado County, California",126849.0,89.4,62.8,62.7,6.0,17.0
4,"Fresno County, California",555181.0,76.6,61.4,59.3,6.0,19.0
5,"Humboldt County, California",90796.0,92.8,58.0,62.8,6.0,23.0
6,"Imperial County, California",105588.0,66.3,53.9,62.8,6.0,25.0
7,"Kern County, California",499760.0,78.9,60.0,59.3,6.0,29.0
8,"Kings County, California",93661.0,78.7,55.8,60.7,6.0,31.0
9,"Lake County, California",46437.0,89.5,57.9,62.9,6.0,33.0


In [32]:
create_male_pop_by_age_df(2019)

,2019
Estimate Total Male,6217305
Estimate Total Male Under 5 years,378778
Estimate Total Male 5 to 9 years,383859
Estimate Total Male 10 to 14 years,420715
Estimate Total Male 15 to 17 years,249124
Estimate Total Male 18 and 19 years,168973
Estimate Total Male 20 years,84744
Estimate Total Male 21 years,86507
Estimate Total Male 22 to 24 years,252191
Estimate Total Male 25 to 29 years,450372


In [22]:
years = [i for i in range(2018, 2020)]
male_pop_by_age_df = pd.DataFrame(columns=['Population Label'])
for year in tqdm(years):
    try:
        y_df = create_male_pop_by_age_df(year).reset_index().rename({'index': 'Population Label'}, axis=1)
        male_pop_by_age_df = pd.merge(male_pop_by_age_df, y_df, how='outer', on='Population Label')
    except IndexError:
        next

  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
male_pop_by_age_df

,Population Label,2018,2019
0,Estimate Total Male,2369271,6217305
1,Estimate Total Male Under 5 years,149436,378778
2,Estimate Total Male 5 to 9 years,151958,383859
3,Estimate Total Male 10 to 14 years,164696,420715
4,Estimate Total Male 15 to 17 years,97945,249124
5,Estimate Total Male 18 and 19 years,72354,168973
6,Estimate Total Male 20 years,38027,84744
7,Estimate Total Male 21 years,35913,86507
8,Estimate Total Male 22 to 24 years,87578,252191
9,Estimate Total Male 25 to 29 years,162850,450372


In [13]:
male_pop_by_age_df.to_csv('data_extract/male_pop_by_age_2005-2019.csv')